# dfs_xgb_with_double_normalization

In [1]:
from draco.demo import load_demo

target_times, readings = load_demo()

In [2]:
pipeline_name = 'dfs_xgb_with_double_normalization'

In [3]:
from draco.pipeline import DracoPipeline

pipeline = DracoPipeline(pipeline_name)

/Users/sarah/opt/anaconda3/envs/draco/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
pipeline.template['primitives']

['mlblocks.MLPipeline',
 'featuretools.dfs',
 'mlprimitives.custom.feature_extraction.CategoricalEncoder',
 'xgboost.XGBClassifier']

# Step by Step execution

## Input Data

In [5]:
readings.head()

turbine_id signal_id  timestamp  value
0       T001       S01 2013-01-10  323.0
1       T001       S02 2013-01-10  320.0
2       T001       S03 2013-01-10  284.0
3       T001       S04 2013-01-10  348.0
4       T001       S05 2013-01-10  273.0

In [6]:
target_times.head()

turbine_id cutoff_time  target
0       T001  2013-01-12       0
1       T001  2013-01-13       0
2       T001  2013-01-14       0
3       T001  2013-01-15       1
4       T001  2013-01-16       0

## Data Preparation (part of Draco Pipeline)

* Input: target_times, readings, turbines
* Output: X, y, readings, turbines
* Effect: target_times has been split into X and y

## mlblocks.MLPipeline

### pandas.DataFrame.resample

* Input: readings
* Output: readings (resampled)
* Effect: readings have been resampled to the indicated resample rule and turbine_id,
  signal_id and timestamp have been set as a multi-index

###  featuretools.EntitySet.entity_from_dataframe

* Input: readings (resampled)
* Output: entityset
* Effect: Entityset has been generated from readings

### featuretools.EntitySet.normalize_entity

* Input: entityset
* Output: entityset with relationship (readings.turbine_id with turbines.turbine_id)
* Effect: establish relation between readings and turbines

### featuretools.EntitySet.normalize_entity

* Input: entityset
* Output: entityset with relationship (readings.signal_id with signals.signal_id)
* Effect: establish relationship between readings and signals

In [7]:
step = 0
context = pipeline.fit(target_times, readings, output_=step)

In [8]:
context.keys()

dict_keys(['readings', 'turbines', 'X', 'y', 'entityset'])

In [9]:
context['entityset']

Entityset: entityset
  Entities:
    readings [Rows: 1329146, Columns: 5]
    turbines [Rows: 1, Columns: 1]
    signals [Rows: 26, Columns: 1]
  Relationships:
    readings.turbine_id -> turbines.turbine_id
    readings.signal_id -> signals.signal_id

## featuretools.dfs

* Input: entityset (unstacked, no turbine_id, no timestamp)
* Output: X (has additional features)
* Effect: build features for relational dataset using DFS

In [10]:
step = 1
context = pipeline.fit(**context, output_=step, start_=step)

In [11]:
context.keys()

dict_keys(['readings', 'turbines', 'entityset', 'X', 'y'])

In [12]:
context['X'].head()

SUM(readings.value)  STD(readings.value)  MAX(readings.value)  \
turbine_id                                                                  
T001               3.433649e+09         1.456860e+06            3448719.0   
T001               3.441489e+09         1.459865e+06            3453777.0   
T001               3.455470e+09         1.465277e+06            3463880.0   
T001               3.475361e+09         1.473337e+06            3474703.0   
T001               2.888083e+09         1.477958e+06            3485019.0   

            SKEW(readings.value)  MIN(readings.value)  MEAN(readings.value)  \
turbine_id                                                                    
T001                    1.019214                  0.0         917107.079193   
T001                    1.018761                  0.0         919201.162179   
T001                    1.018192                  2.7         922935.352244   
T001                    1.017666                 -1.0         928248.092869   
T001                    1.032002                  0.0         924186.531200   

            COUNT(readings)  NUM_UNIQUE(readings.signal_id)  \
turbine_id                                                    
T001                   3744                              26   
T001                   3744                              26   
T001                   3744                              26   
T001                   3744                              26   
T001                   3744                              26   

           MODE(readings.signal_id)  NUM_UNIQUE(readings.DAY(timestamp))  ...  \
turbine_id                                                                ...   
T001                            S01                                    2  ...   
T001                            S01                                    2  ...   
T001                            S01                                    2  ...   
T001                            S01                                    2  ...   
T001                            S01                                    2  ...   

            MEAN(readings.signals.NUM_UNIQUE(readings.WEEKDAY(timestamp)))  \
turbine_id                                                                   
T001                                                      2.0                
T001                                                      2.0                
T001                                                      2.0                
T001                                                      2.0                
T001                                                      2.0                

            MEAN(readings.signals.NUM_UNIQUE(readings.YEAR(timestamp)))  \
turbine_id                                                                
T001                                                      1.0             
T001                                                      1.0             
T001                                                      1.0             
T001                                                      1.0             
T001                                                      1.0             

            NUM_UNIQUE(readings.signals.MODE(readings.DAY(timestamp)))  \
turbine_id                                                               
T001                                                        1            
T001                                                        1            
T001                                                        1            
T001                                                        1            
T001                                                        1            

            NUM_UNIQUE(readings.signals.MODE(readings.YEAR(timestamp)))  \
turbine_id                                                                
T001                                                        1             
T001                                                        1             
T001            

In [13]:
# features generated (the turbine_id is set as index).
len(context['X'].columns)

99

In [14]:
context['readings'].head()

turbine_id signal_id  timestamp  value
0       T001       S01 2013-01-10  323.0
1       T001       S02 2013-01-10  320.0
2       T001       S03 2013-01-10  284.0
3       T001       S04 2013-01-10  348.0
4       T001       S05 2013-01-10  273.0

## mlprimitives.custom.feature_extraction.CategoricalEncoder

* Input: X
* Output: X (label encoded)
* Effect: encodes categorical features using OneHotLabelEncoder

In [15]:
step = 2
context = pipeline.fit(**context, output_=step, start_=step)

In [16]:
context.keys()

dict_keys(['readings', 'turbines', 'entityset', 'X', 'y'])

In [17]:
context['X'].head()

SUM(readings.value)  STD(readings.value)  MAX(readings.value)  \
turbine_id                                                                  
T001               3.433649e+09         1.456860e+06            3448719.0   
T001               3.441489e+09         1.459865e+06            3453777.0   
T001               3.455470e+09         1.465277e+06            3463880.0   
T001               3.475361e+09         1.473337e+06            3474703.0   
T001               2.888083e+09         1.477958e+06            3485019.0   

            SKEW(readings.value)  MIN(readings.value)  MEAN(readings.value)  \
turbine_id                                                                    
T001                    1.019214                  0.0         917107.079193   
T001                    1.018761                  0.0         919201.162179   
T001                    1.018192                  2.7         922935.352244   
T001                    1.017666                 -1.0         928248.092869   
T001                    1.032002                  0.0         924186.531200   

            COUNT(readings)  NUM_UNIQUE(readings.signal_id)  \
turbine_id                                                    
T001                   3744                              26   
T001                   3744                              26   
T001                   3744                              26   
T001                   3744                              26   
T001                   3744                              26   

            NUM_UNIQUE(readings.DAY(timestamp))  \
turbine_id                                        
T001                                          2   
T001                                          2   
T001                                          2   
T001                                          2   
T001                                          2   

            NUM_UNIQUE(readings.MONTH(timestamp))  ...  \
turbine_id                                         ...   
T001                                            1  ...   
T001                                            1  ...   
T001                                            1  ...   
T001                                            1  ...   
T001                                            1  ...   

            NUM_UNIQUE(readings.signals.MODE(readings.DAY(timestamp)))  \
turbine_id                                                               
T001                                                        1            
T001                                                        1            
T001                                                        1            
T001                                                        1            
T001                                                        1            

            NUM_UNIQUE(readings.signals.MODE(readings.YEAR(timestamp)))  \
turbine_id                                                                
T001                                                        1             
T001                                                        1             
T001                                                        1             
T001                                                        1             
T001                                                        1             

            NUM_UNIQUE(readings.signals.MODE(readings.WEEKDAY(timestamp)))  \
turbine_id                                                                   
T001                                                        1                
T001                                                        1                
T001                                                        1                
T001                                                        1                
T001                                                        1                

            NUM_UNIQUE(readings.signals.MODE(readings.MONTH(timestamp)))  \
turbine_id                                 

In [18]:
context['readings'].head()

turbine_id signal_id  timestamp  value
0       T001       S01 2013-01-10  323.0
1       T001       S02 2013-01-10  320.0
2       T001       S03 2013-01-10  284.0
3       T001       S04 2013-01-10  348.0
4       T001       S05 2013-01-10  273.0

## xgboost.XGBClassifier

* Input: X (label encoded and featurized)
* Output: None
* Effect: trained model

In [19]:
step = 3
context = pipeline.fit(**context, output_=step, start_=step)

In [20]:
context.keys()

dict_keys(['readings', 'turbines', 'entityset', 'X', 'y'])